In [ ]:
!python --version


Python 3.10.12


In [ ]:
import torch
print(torch.__version__)

2.5.1+cu121


In [1]:
!pip install  ultralytics==8.3.29  roboflow==1.1.48 supervision==0.25.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.8/883.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive


/content/drive/MyDrive


In [3]:
!pip install fastapi uvicorn python-multipart


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.5 MB/s eta 0:00:00


In [4]:
code = """
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
from ultralytics import YOLO
from PIL import Image
import json
import io

# Initialize YOLOv8 model
model_path = '/content/drive/MyDrive/best.pt'
model = YOLO(model_path)

# FastAPI app setup
app = FastAPI()

class PredictionResponse(BaseModel):
    predictions: list

@app.post("/predict", response_model=PredictionResponse)
async def predict(file: UploadFile = File(...)):
    # Load image from incoming file
    image_bytes = await file.read()
    img = Image.open(io.BytesIO(image_bytes)).convert("RGB")

    # Perform inference
    results = model(img)

    # Process results
    predictions = []
    for box in results[0].boxes:
        predictions.append({
            "class": int(box.cls),  # Class ID
            "label": model.names[int(box.cls)],  # Class label
            "confidence": float(box.conf),  # Confidence score
            "bbox": [float(coord) for coord in box.xyxy[0]]  # Convert tensor to list of floats
        })

    # Return predictions
    return PredictionResponse(predictions=predictions)
"""

# Save the code as a Python file
with open("/content/main.py", "w") as f:
    f.write(code)


In [5]:
!pip install pyngrok

In [6]:
from pyngrok import ngrok


In [7]:
!ngrok config add-authtoken 2p4cVBfLlSEkDsrFkEtn80jhm9Y_6bxmQovWkhhRXvR56bnKC


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
# Open a tunnel to the FastAPI app
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://adee-35-229-254-47.ngrok-free.app" -> "http://localhost:8000"


In [9]:
!uvicorn main:app --host 0.0.0.0 --port 8000 --reload


INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [1209] using StatReload
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
INFO:     Started server process [1211]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     197.43.18.1:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     197.43.18.1:0 - "GET /openapi.json HTTP/1.1" 200 OK

0: 512x768 2 Persons, 3 vehicles, 4459.5ms
Speed: 29.9ms preprocess, 4459.5ms inference, 34.1ms postprocess per image at shape (1, 3, 512, 768)
INFO:     197.43.18.1:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     Shutting down
INFO:     Waiting for ap